In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('../data/eval/spljosteni_sa_ekonomijom.pkl')
df.set_index('instance_id', inplace=True)

df.head()

,KLIJENT_ID,OZNAKA_PARTIJE,PRVI_DATUM_OTVARANJA,ZADNJI_DATUM_OTVARANJA,PRVI_PLANIRANI_DATUM_ZATVARANJA,ZADNJI_PLANIRANI_DATUM_ZATVARANJA,VRSTA_KLIJENTA,STAROST,VRSTA_PROIZVODA,PROIZVOD,...,TREND_nezaposlenosti,TREND_nafta,STD_bdp,STD_inflacija,STD_nezaposlenosti,STD_nafta,PROSJEK_bdp,PROSJEK_inflacija,PROSJEK_nezaposlenosti,PROSJEK_nafta
instance_id,,,,,,,,,,,,,,,,,,,,,
0,1049000,5585453,2012-07-14,2012-07-14,2013-07-15,2013-07-15,1410,64,L,TM0109,...,-1.2,-5.44,0.90000,0.60000,0.600000,2.720,0.9,-0.600000,0.60,2.720
4,14884,1183149,2006-03-23,2011-03-27,2011-03-27,2012-03-28,1410,53,L,TM0109,...,-1.2,16.17,1.00000,0.55000,0.600000,8.085,-1.0,0.550000,0.60,-8.085
5,1348702,12847410,2018-07-20,2018-08-04,2020-02-15,2020-02-15,1410,35,L,TM0109,...,1.0,4.03,0.08165,0.04714,0.816497,2.015,-0.1,0.033333,-1.00,-2.015
6,764915,5647773,2013-12-21,2013-12-21,2014-03-22,2014-03-22,1410,50,L,TM0109,...,0.5,-10.74,0.20000,1.20000,0.250000,5.370,0.2,-1.200000,-0.25,5.370
7,367476,4875789,2010-01-08,2010-01-08,2011-01-09,2011-01-09,1410,35,L,TM0109,...,-0.2,-28.73,0.60000,0.65000,0.100000,14.365,0.6,0.650000,0.10,14.365


In [3]:
df.shape[0]

76605

In [4]:
df.drop(columns=['KLIJENT_ID', 'OZNAKA_PARTIJE', 'STAROST', 'PRIJEVREMENI_RASKID'], axis = 1, inplace = True)
# df = df[(df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.year < 2019) | ((df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.year == 2019) & (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.month == 1) & (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'].dt.day <= 10))]


## pocisti ZADNJI_UGOVORENI_IZNOS == 0 jer to ne ulazi u evaluaciju na internetu
df = df[df['ZADNJI_UGOVORENI_IZNOS'] != 0]

In [5]:
df.loc[pd.isnull(df['PRVA_VISINA_KAMATE']) | pd.isnull(df['ZADNJA_VISINA_KAMATE']), 'PRVA_VISINA_KAMATE'] = 0
df.loc[pd.isnull(df['PRVA_VISINA_KAMATE']) | pd.isnull(df['ZADNJA_VISINA_KAMATE']), 'ZADNJA_VISINA_KAMATE'] = 0

df[pd.isnull(df['PRVA_VISINA_KAMATE']) | pd.isnull(df['ZADNJA_VISINA_KAMATE'])].shape[0]

0

In [6]:
df['zadnja_duljina_trajanja'] = (df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'] - df['ZADNJI_DATUM_OTVARANJA']).dt.days

df['PRVI_DATUM_OTVARANJA'] = pd.to_numeric(df['PRVI_DATUM_OTVARANJA']) / 10e11
df['PRVI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['PRVI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11
df['ZADNJI_DATUM_OTVARANJA'] = pd.to_numeric(df['ZADNJI_DATUM_OTVARANJA']) / 10e11
df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['ZADNJI_PLANIRANI_DATUM_ZATVARANJA']) / 10e11

In [7]:
# df['promjena_valute'] = (~(df['PRVA_VALUTA'] == df['ZADNJA_VALUTA'])).astype(int)
# df.drop('PRVA_VALUTA', axis=1, inplace=True)

df['promjena_tipa_kamate'] = (~(df['PRVI_TIP_KAMATE'] == df['ZADNJI_TIP_KAMATE'])).astype(int)
df.drop('PRVI_TIP_KAMATE', axis=1, inplace=True)

df['promjena_visine_kamate'] = df['ZADNJA_VISINA_KAMATE'] - df['PRVA_VISINA_KAMATE']
df.drop('PRVA_VISINA_KAMATE', axis=1, inplace=True)

df['promjena_ugovorenog_iznosa'] = df['ZADNJI_UGOVORENI_IZNOS'] - df['PRVI_UGOVORENI_IZNOS']
df.drop('PRVI_UGOVORENI_IZNOS', axis=1, inplace=True)


df['ukupni_dug_po_danu'] = (df['ZADNJI_UGOVORENI_IZNOS'] + (df['ZADNJI_UGOVORENI_IZNOS'] * df['ZADNJA_VISINA_KAMATE'] * df['zadnja_duljina_trajanja'])/36500 ) / df['zadnja_duljina_trajanja']
df['zadnji_ugovoreni_iznos_log'] = np.log(df['ZADNJI_UGOVORENI_IZNOS'])

df.sample(12)

,PRVI_DATUM_OTVARANJA,ZADNJI_DATUM_OTVARANJA,PRVI_PLANIRANI_DATUM_ZATVARANJA,ZADNJI_PLANIRANI_DATUM_ZATVARANJA,VRSTA_KLIJENTA,VRSTA_PROIZVODA,PROIZVOD,ZADNJI_UGOVORENI_IZNOS,PRVA_VALUTA,ZADNJA_VALUTA,...,PROSJEK_bdp,PROSJEK_inflacija,PROSJEK_nezaposlenosti,PROSJEK_nafta,zadnja_duljina_trajanja,promjena_tipa_kamate,promjena_visine_kamate,promjena_ugovorenog_iznosa,ukupni_dug_po_danu,zadnji_ugovoreni_iznos_log
instance_id,,,,,,,,,,,,,,,,,,,,,
77610,1271980.8,1271980.8,1367280.0,1367280.0,1410,A,FL1200,55875.58,1,1,...,-0.350000,-1.225000,-1.050000,-14.822500,1103,0,0.0,0.0,63.669945,10.930883
37110,1401235.2,1401235.2,1480464.0,1480464.0,1410,A,FL0801,20000.00,1,1,...,-2.033333,0.400000,2.166667,32.006667,917,0,0.0,0.0,26.303402,9.903488
78143,1166140.8,1166140.8,1293753.6,1293753.6,1410,A,FL1200,16852.00,4,4,...,4.220000,0.080000,1.320000,-9.296000,1477,0,0.0,0.0,14.987778,9.732225
3191,1394496.0,1394496.0,1402531.2,1402531.2,1410,L,TM0109,48976.80,1,1,...,2.033333,-0.400000,-2.166667,-32.006667,93,0,0.0,0.0,529.852650,10.799102
6932,1484697.6,1484697.6,1516320.0,1516320.0,1550,L,TM0109,26506.93,2,2,...,-0.100000,0.200000,-1.200000,9.360000,366,0,0.0,0.0,72.822726,10.185161
45133,1349827.2,1349827.2,1667174.4,1667174.4,1410,A,FL0801,131500.00,1,1,...,-3.963636,2.218182,5.581818,15.407500,3673,0,0.0,0.0,64.263441,11.786762
64837,1387411.2,1387411.2,1398816.0,1398816.0,1410,A,FL1100,333.70,1,1,...,-0.200000,1.200000,0.250000,-5.370000,132,0,0.0,0.0,2.528030,5.810242
74778,1453334.4,1453334.4,1466467.2,1466467.2,1120,A,FL0900,700000.00,1,1,...,0.466667,-1.600000,2.533333,-5.720000,152,0,0.0,0.0,4685.811103,13.458836
59919,1337126.4,1337126.4,1496188.8,1496188.8,1610,A,FL0803,37500.93,2,2,...,-3.283333,2.583333,1.566667,13.375000,1841,0,0.0,0.0,27.048117,10.532121


In [8]:
df.shape[0]

76579

In [9]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.load_model('nas_model1')

In [10]:
df['PRIJEVREMENI_RASKID'] = pd.Series(model.predict(data=df).astype(int), index=df.index)

print((df['PRIJEVREMENI_RASKID'] == 0).sum())
print((df['PRIJEVREMENI_RASKID'] == 1).sum())

20599
55980


In [12]:
predicted = df[df['PRIJEVREMENI_RASKID'] == 1]
predicted = predicted.loc[:, df.columns.intersection(['PRIJEVREMENI_RASKID'])]

In [13]:
predicted.head()

,PRIJEVREMENI_RASKID
instance_id,
5,1
8,1
17,1
25,1
32,1


In [14]:
original = pd.read_csv('../data/eval/original_sa_nulama.csv')
original.head()

,Unnamed: 0,Unnamed: 0.1,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,0,0,1049000,5585453,2012-07-14,2013-07-15,273555.01,2,1410,TM0109,L,3.60,A,64,0
1,1,1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,0
2,2,2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,0
3,3,3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,0
4,4,4,4,14884,1183149,2011-03-27,2012-03-28,45315.31,2,1410,TM0109,L,3.30,C,53,0


In [15]:
# for index in predicted.index.values:
#     original.loc[original['instance_id']==index, 'PRIJEVREMENI_RASKID'] = 1
original.loc[predicted.index, 'PRIJEVREMENI_RASKID'] = predicted

In [16]:
original.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [17]:
original['PRIJEVREMENI_RASKID'] = original['PRIJEVREMENI_RASKID'].map({1: 'Y', 0: 'N'})

original.head(20)

,instance_id,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID
0,0,1049000,5585453,2012-07-14,2013-07-15,273555.01,2,1410,TM0109,L,3.60,A,64,N
1,1,481302,752335,2016-01-24,2016-01-24,50970.78,2,1410,TM0109,L,0.20,A,56,N
2,2,838192,6623548,2018-12-26,2018-12-26,8969.36,2,1410,TM0109,L,0.02,A,52,N
3,3,174575,425060,2012-08-04,2012-08-04,16142.48,2,1410,TM0109,L,2.30,C,76,N
4,4,14884,1183149,2011-03-27,2012-03-28,45315.31,2,1410,TM0109,L,3.30,C,53,N
5,5,1348702,12847410,2018-08-04,2020-02-15,40000.00,1,1410,TM0109,L,1.50,A,35,Y
6,6,764915,5647773,2013-12-21,2014-03-22,119043.94,2,1410,TM0109,L,2.00,A,50,N
7,7,367476,4875789,2010-01-08,2011-01-09,103650.71,2,1410,TM0109,L,5.15,A,35,N
8,8,234350,12636299,2017-08-08,2018-08-09,185000.00,1,1550,TM0109,L,1.30,A,46,Y
9,9,888283,9088014,2018-11-23,2018-11-23,54123.73,1,1410,TM0109,L,0.20,A,38,N


In [18]:
original.to_csv('../data/student.csv')

In [19]:
original.shape[0]

84903